## Recomendador básico

Instalar el Frameword Surprise

In [2]:
!pip install numpy
!pip install scikit-surprise

Importamos las bibliotecas necesarias para nuestro recomendador

In [3]:
# Cargamos las librerías necesarias de Pandas
import pandas as pd
import numpy as np

In [4]:
# Cargamos las librerías necesarias de Surprise
from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import accuracy

Cargamos el dataset en pandas para luego pasarlo a surprise

In [12]:
# Cargamos el CSV con el dataframe de Pandas
df = pd.read_csv('spotify_data.csv')

In [13]:
df

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.694000,0.000000,0.1150,0.1390,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.477000,0.000014,0.0974,0.5150,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.338000,0.000050,0.0895,0.1450,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.807000,0.000000,0.0797,0.5080,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.072600,0.019300,0.1100,0.2170,171.864,244320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193605,193605,Periphery,The Event,1fPfAhG2nskq3gDhh0Z5fT,26,2015,metal,0.226,0.560,7,-6.772,1,0.0326,0.001820,0.865000,0.1980,0.0682,126.021,108228,4
193606,193606,Between The Buried And Me,Famine Wolf,6IeXWJ2CKX8Dnir6pRk6U0,31,2015,metal,0.295,0.956,1,-4.945,1,0.1090,0.000063,0.000155,0.1330,0.3070,125.152,410827,4
193607,193607,Parkway Drive,Vicious,6YvdktHhWF5vNA8oHKGFq4,32,2015,metal,0.499,0.980,11,-3.573,0,0.0988,0.000037,0.270000,0.1980,0.1840,126.988,261571,4
193608,193608,Born Of Osiris,Illuminate,4v7XuavJGvFGG8KRQqOGnT,30,2015,metal,0.435,0.983,2,-3.308,1,0.2210,0.000378,0.000000,0.1770,0.2190,155.036,285677,4


In [14]:
reader = Reader(rating_scale=(1, 30))

# Load the Pandas DataFrame into a Surprise Dataset
data = Dataset.load_from_df(df[['artist_name', 'track_name', 'valence']], reader)

Entrenar los datos

In [15]:
# Crear un modelo modelo k-NN
sim_options = {
    'name': 'cosine',  
    'user_based': False  # Recomendador basado en contenidos
}
clf = KNNBasic(sim_options=sim_options, k=50, verbose=True)

In [16]:
#Measures probar con RMSE o MAE, cuál sea el mejor
cv = cross_validate(clf, data, measures=['RMSE', 'MAE'], cv=7, verbose=True)

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 157. GiB for an array with shape (144993, 144993) and data type float64

Función para determinar canciones similares

In [9]:
#Para hacer la búsqueda si la canción está en los datos entrenados
items_ids = [clf.trainset.to_raw_iid(iid) for iid in clf.trainset.all_items()]

In [10]:
def get_similar_songs(song_title, k=10):
    try: 
        # Comprueba si la canción está en los datos entrenados
        if song_title not in items_ids:
            return []  # Devuelve una lista vacía si no está entre los datos entrenados

        # Obtiene el id de la canción para luego pasarlo al algoritmo
        song_id = clf.trainset.to_inner_iid(song_title)

        # Usa el modelo k-NN para encontrar canciones similares
        similar_items = clf.get_neighbors(song_id, k)

        # Una vez tiene los ids de las canciones similares se pasa a los nombres reales de las canciones
        similar_songs = [clf.trainset.to_raw_iid(item_id) for item_id in similar_items]

        return similar_songs

    except IndexError as e:
        return []

Generar una "playlist" random con 10 canciones

In [7]:
import random

random_songs = df.sample(n=10)

In [ ]:
random_songs

Conseguir las recomendaciones 

In [11]:
similar_songs = []

for song_title in random_songs['title']:
    sim_songs = get_similar_songs(song_title, k=10)    
    
    #Para evitar canciones ya recomendadas antes
    for s in sim_songs:
        if s not in similar_songs:
            similar_songs.append(s)

In [12]:
similar_songs

['Walk On Water (feat. Beyoncé)',
 'Grenade',
 'Hey Brother',
 'TiK ToK',
 'Uptown Funk',
 'Mark My Words',
 'Yeah 3x',
 'Pom Poms',
 'Sheezus',
 'Wish That You Were Here - From “Miss Peregrine’s Home for Peculiar Children” Original Motion Picture',
 'Never Be the Same - Radio Edit',
 'Crying in the Club',
 'Good Life',
 'No Vacancy (with Sebastián Yatra)',
 'Secrets',
 'If I Lose Myself - Alesso vs OneRepublic',
 'I Lived',
 'Rich Love (with Seeb)',
 'All The Right Moves',
 'Wherever I Go',
 'Kill Em With Kindness',
 'Moment 4 Life - Album Version (Edited)',
 'Starships',
 'Chun-Li',
 'Anaconda',
 'Love The Way You Lie',
 'Work',
 'S&M Remix',
 'Hard',
 'Love On The Brain',
 'Only Girl (In The World)',
 "What's My Name?",
 'Where Have You Been',
 'Diamonds',
 'Right Now - Dyro Radio Edit',
 'We Found Love',
 'Wait',
 'Moves Like Jagger - Studio Recording From The Voice Performance',
 'Cold (feat. Future)',
 'Sugar',
 'Love Somebody',
 'Maps',
 'Misery',
 'Girls Like You (feat. Cardi B

Vamos a probar recomendación con una canción

In [13]:
similar_songs = get_similar_songs('Find U Again (feat. Camila Cabello)', k=10)
similar_songs

['Uptown Funk',
 'Nothing Breaks Like a Heart (feat. Miley Cyrus)',
 'Grenade',
 'Hey Brother',
 'TiK ToK',
 'Mark My Words',
 'Yeah 3x',
 'Pom Poms',
 'Sheezus',
 'Wish That You Were Here - From “Miss Peregrine’s Home for Peculiar Children” Original Motion Picture']

#### Recomendador sin surprise
Dado una canción o el nombre de un artista, se recomienda más canciones del artista (en el caso de por canción no se recomienda esa canción)

In [18]:
df = df.sort_values(by='popularity') # ordena por popularidad
df2_grouped = df.groupby('artist_name') # pone las canciones del mismo artista juntos
df3_grouped = df.groupby('genre') # por géneros porque en el caso de que no haya suficientes del mismo artista será por las canciones más populares del género

df2 = pd.DataFrame()
for key, group in df2_grouped:
    df2 = pd.concat([df2, group], axis=0)

df3 = pd.DataFrame()
for key, group in df3_grouped:
    df3 = pd.concat([df3, group], axis=0)

In [20]:
df2.loc[df2['genre'] == "pop"]

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
93719,93719,2CELLOS,We Found Love,0bt22Pke8uyTr7az9pseW9,51,2013,pop,0.332,0.399,2,-11.316,1,0.0447,0.79700,0.886000,0.5010,0.589,129.900,212600,4
93759,93759,7!!,ラヴァーズ,5Do3qmFMpCjVLmtLhmj3a1,57,2013,pop,0.492,0.953,11,-2.683,1,0.1450,0.00363,0.000000,0.3210,0.647,163.783,222200,4
40623,40623,ABBA,One Of Us,6zgtBUEkAfilJ2YEOvNexR,52,2012,pop,0.646,0.388,7,-15.264,1,0.0388,0.49500,0.000634,0.0673,0.854,82.216,237720,4
40529,40529,ABBA,Angeleyes,1zZLRpTzRTqPQ7G7uxYI9Y,57,2012,pop,0.711,0.827,11,-10.809,1,0.0331,0.49700,0.000254,0.0822,0.964,133.157,262493,4
40438,40438,ABBA,Dancing Queen,1rMfDvE2C8ne8UZj847rKM,62,2012,pop,0.508,0.825,9,-9.627,1,0.0451,0.17500,0.001330,0.3760,0.715,100.548,233440,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40563,40563,fun.,Why Am I the One,5QbQ5iAebksB5Wj5BPazNX,43,2012,pop,0.408,0.499,6,-7.160,1,0.0547,0.44000,0.000004,0.5950,0.501,82.157,286707,4
146982,146982,fun.,Sight of the Sun - Single Version,4orHVYvdG5v4G4bmp2Lwdg,47,2014,pop,0.626,0.703,11,-5.247,1,0.0306,0.02750,0.000002,0.0903,0.522,92.054,210373,4
40379,40379,fun.,Carry On,7gpy7sfWPNuOKmUNs3XQYE,58,2012,pop,0.388,0.694,5,-5.769,1,0.0735,0.11800,0.000293,0.0820,0.365,145.434,278373,5
40353,40353,fun.,Some Nights,6t6oULCRS6hnI7rm0h5gwl,68,2012,pop,0.672,0.738,0,-7.045,1,0.0506,0.01780,0.000068,0.0927,0.392,107.938,277040,4


In [21]:
def recommender_by_artist(artist, song_excl):
    songs = df2.loc[df2['artist_name'] == artist]
    if song_excl != None: 
        songs_ret = songs.loc[songs['track_name'] != song_excl, ['track_id', 'track_name', 'artist_name', 'genre']]
        return songs_ret
    else: 
        return songs[['track_id', 'track_name', 'artist_name', 'genre']]

In [22]:
def recommender_by_genre(genre, song_excl):
    songs = df3.loc[df3['genre'] == genre]
    if song_excl != None: 
        songs_ret = songs.loc[songs['track_name'] != song_excl, ['track_id', 'track_name', 'artist_name', 'genre']]
        return songs_ret
    else: 
        return songs[['track_id', 'track_name', 'artist_name', 'genre']]

In [23]:
def get_id(songs):
    return songs['track_id']  

In [24]:
def recommender_manual(song_artist):
    if song_artist in df['track_id'].values: #lo que se ha pasado por parámetro es el título de la canción
        fil = df.loc[df['track_id'] == song_artist]
        artist = fil['artist_name'].values[0]
        song_name = fil['track_name'].values[0]
        
        songs = recommender_by_artist(artist, song_name)

        if len(songs) < 10:
            genre = fil['genre'].values[0] 
            songs_genre = recommender_by_genre(genre, song_name)
            
            songs_ret = pd.concat([songs, songs_genre])[:10]

            return get_id(songs_ret)
        
        elif len(songs) > 10:
            return get_id(songs[:10])
        
        else:
            return get_id(songs)

    elif song_artist in df['artist_name'].values: #se ha pasado por parámetro el nombre del artista
        songs = recommender_by_artist(song_artist, None)

        return get_id(songs[:10]) #no hay genero asociado por artista, lo que muestre si hay 15 o menos del artista
    
    else: #no exista tal artista o canción
        print("Error: no existe tal artista o canción en la base de datos\n")
        return []

In [25]:
def recommender_no_surprise(songs_arists):
    songs = []
    for song_artist in songs_arists:
        songs_aux = recommender_manual(song_artist)
        songs.extend(songs_aux)
    
    return songs

In [26]:
recommender_manual('4orHVYvdG5v4G4bmp2Lwdg') # "#thatPower"

43957    4nRz4oj8uq2iEkUFaIKY4R
43938    15CBXNDYxdrtnZ9ILYAOXe
43924    1JAI5Ia020mdGH2wMQEacy
43925    23wTcSwRHu8E513oeri4xK
43926    7c6TRb4cvgHWVhTJDfRqth
43944    5tbz0VnVwfF3jubvjDGdHo
43880    47XL5AzqSH1SsOEl7QSvXW
43871    3xsMa7CA11iRjhmIdEy2N6
40563    5QbQ5iAebksB5Wj5BPazNX
40379    7gpy7sfWPNuOKmUNs3XQYE
Name: track_id, dtype: object

In [27]:
recommender_manual('ABBA')

52266    710gAFAOwtkoDvXRfszfwY
52265    2qIrqbIvlrzKaA3jIQMTxa
52251    3mES4dPHPHx76TlnTiIbTR
52239    5P4DBjoQdju1tjv3wP4oSD
52233    2O8mjTtmG2aHixvZfKYMor
52240    0oZObEr6GJ8bnf6fs6w12G
52230    0rM6L2iVmThaLzTP9Oqoss
52224    0K5XQmDt8HCpVMVmKBsKk2
52209    7Bdl6afVfWgQZ0CN89ZxoG
52222    7e4Hyk35hXl7xcF3PRXa26
Name: track_id, dtype: object

In [28]:
recommender_no_surprise(['4orHVYvdG5v4G4bmp2Lwdg', 'ABBA'])

['4nRz4oj8uq2iEkUFaIKY4R',
 '15CBXNDYxdrtnZ9ILYAOXe',
 '1JAI5Ia020mdGH2wMQEacy',
 '23wTcSwRHu8E513oeri4xK',
 '7c6TRb4cvgHWVhTJDfRqth',
 '5tbz0VnVwfF3jubvjDGdHo',
 '47XL5AzqSH1SsOEl7QSvXW',
 '3xsMa7CA11iRjhmIdEy2N6',
 '5QbQ5iAebksB5Wj5BPazNX',
 '7gpy7sfWPNuOKmUNs3XQYE',
 '710gAFAOwtkoDvXRfszfwY',
 '2qIrqbIvlrzKaA3jIQMTxa',
 '3mES4dPHPHx76TlnTiIbTR',
 '5P4DBjoQdju1tjv3wP4oSD',
 '2O8mjTtmG2aHixvZfKYMor',
 '0oZObEr6GJ8bnf6fs6w12G',
 '0rM6L2iVmThaLzTP9Oqoss',
 '0K5XQmDt8HCpVMVmKBsKk2',
 '7Bdl6afVfWgQZ0CN89ZxoG',
 '7e4Hyk35hXl7xcF3PRXa26']